### Import dependencies

In [1]:
%load_ext autotime

from IPython.display import display_html
from PIL import Image as PILImage
from openpyxl import load_workbook
from io import BytesIO
import cv2

time: 297 ms (started: 2023-12-16 17:32:42 +01:00)


### Documents

Documents can be either an image or a PDF file. Check README for supported formats.

#### Image

In [2]:
from img2table.document import Image

img_path = "data/tables.png"

# Definition of image from path
img_from_path = Image(src=img_path)

# Definition of image from bytes
with open(img_path, 'rb') as f:
    img_bytes = f.read()
img_from_bytes = Image(src=img_bytes)

# Definition of image from file-like object
img_from_file_like = Image(src=BytesIO(img_bytes))

time: 500 ms (started: 2023-12-16 17:32:44 +01:00)


#### PDF

In [3]:
from img2table.document import PDF

pdf_path = "data/tables.pdf"

# Definition of PDF from path
# The optional pages argument enables the extraction of table on specific pages of the PDF
pdf_from_path = PDF(src=pdf_path, pages=[0, 1])

# Definition of PDF from bytes
with open(pdf_path, 'rb') as f:
    pdf_bytes = f.read()
pdf_from_bytes = PDF(src=pdf_bytes)

# Definition of PDF from file-like object
pdf_from_file_like = PDF(src=BytesIO(pdf_bytes))

time: 16 ms (started: 2023-12-16 17:32:46 +01:00)


### OCR instances

<code>img2table</code> provides supports for several OCR tools and services :
  * Tesseract
  * PaddleOCR
  * EasyOCR
  * docTR
  * Google Vision
  * AWS Textract
  * Azure Cognitive Services
  
For all OCRs except Tesseract, a specific installation is needed. Check README for relevant instructions.

In [4]:
# Tesseract OCR
from img2table.ocr import TesseractOCR

tesseract_ocr = TesseractOCR(n_threads=1, lang="eng")

# PaddleOCR
from img2table.ocr import PaddleOCR

paddle_ocr = PaddleOCR(lang="en", kw={"use_dilation": True})

# EasyOCR
from img2table.ocr import EasyOCR

easyocr = EasyOCR(lang=["en"], kw={"gpu": False})

# docTR
from img2table.ocr import DocTR

doctr = DocTR(detect_language=True, kw={"detect_orientation": True})

# Google Vision OCR
from img2table.ocr import VisionOCR

vision_ocr = VisionOCR(api_key="***")

# AWS Textract OCR
from img2table.ocr import TextractOCR

textract_ocr = TextractOCR(aws_access_key_id="***",
                           aws_secret_access_key="***",
                           aws_session_token="***",
                           region="eu-west-1")

# Azure Cognitive Services OCR
from img2table.ocr import AzureOCR

azure_ocr = AzureOCR(endpoint="abc.azure.com",
                           subscription_key="***")

time: 10.2 s (started: 2023-12-16 17:32:47 +01:00)


### Table identification

For a document object (image or pdf), the <code>extract_tables</code> method can be called to perform table identification.<br>
A list/dict of <code>ExtractedTable</code> objects is returned.

In [5]:
img = Image(src="data/tables.png")

# Extract tables
extracted_tables = img.extract_tables()

time: 94 ms (started: 2023-12-16 17:32:58 +01:00)


In [6]:
# Display extracted tables
table_img = cv2.imread("data/tables.png")

for table in extracted_tables:
    for row in table.content.values():
        for cell in row:
            cv2.rectangle(table_img, (cell.bbox.x1, cell.bbox.y1), (cell.bbox.x2, cell.bbox.y2), (255, 0, 0), 2)
            
PILImage.fromarray(table_img)

time: 16 ms (started: 2023-12-16 17:32:58 +01:00)


### Table extraction

If an ocr instance is provided to the <code>extract_table</code> method, cells content will be infered from the OCR result.<br>
A <code>pd.DataFrame</code> representation of the extracted tables can be accessed via the <code>df</code> attribute.

In [7]:
pdf = PDF(src="data/tables.pdf")

# Extract tables
extracted_tables = pdf.extract_tables(ocr=tesseract_ocr,
                                      implicit_rows=False,
                                      borderless_tables=False,
                                      min_confidence=50)

extracted_tables

{0: [ExtractedTable(title=Example of Data Table 1, bbox=(235, 249, 1442, 543),shape=(5, 4)),
  ExtractedTable(title=Example of Data Table 2, bbox=(236, 671, 1451, 971),shape=(5, 4))],
 1: [ExtractedTable(title=Example of Data Table 3, bbox=(235, 249, 1442, 543),shape=(5, 4)),
  ExtractedTable(title=Example of Data Table 4, bbox=(236, 671, 1451, 971),shape=(5, 4))]}

time: 562 ms (started: 2023-12-16 17:33:03 +01:00)


In [8]:
for page, tables in extracted_tables.items():
    for idx, table in enumerate(tables):
        display_html(table.html_repr(title=f"Page {page + 1} - Extracted table n°{idx + 1}"), raw=True)

Page 1 - Extracted table n°1 
 
 Title: Example of Data Table 1 
 Bounding box: x1=235, y1=249, x2=1442, y2=543
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 sample 
 blue LED value 
 green LED value 
 red LED value 
 
 
 1 
 clear water 
 97 
 19 
 79 
 
 
 2 
 blue water 
 73 
 11 
 13 
 
 
 3 
 green water 
 35 
 15 
 14 
 
 
 4 
 tea water 
 33 
 13 
 70

Page 1 - Extracted table n°2 
 
 Title: Example of Data Table 2 
 Bounding box: x1=236, y1=671, x2=1451, y2=971
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 sample 
 blue % transmitted 
 green % transmitted 
 red % transmitted 
 
 
 1 
 clear water 
 97/97 = 100% 
 19/19 = 100% 
 79/79 = 100% 
 
 
 2 
 blue water 
 73/97 = 75% 
 11/19 = 58% 
 13/79 = 17% 
 
 
 3 
 green water 
 35/97 = 36% 
 15/19 = 79% 
 14/79 = 18% 
 
 
 4 
 tea water 
 33/97 = 34% 
 13/19 = 68% 
 70/79 = 91%

Page 2 - Extracted table n°1 
 
 Title: Example of Data Table 3 
 Bounding box: x1=235, y1=249, x2=1442, y2=543
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 sample 
 blue LED value 
 green LED value 
 red LED value 
 
 
 1 
 + 1 tsp milk 
 13 
 14 
 12 
 
 
 2 
 +1tsp milk + blue 
 10 
 9 
 6 
 
 
 3 
 +1tsp milk +green 
 9 
 13 
 7 
 
 
 4 
 +1tsp milk + tea 
 7 
 9 
 9

Page 2 - Extracted table n°2 
 
 Title: Example of Data Table 4 
 Bounding box: x1=236, y1=671, x2=1451, y2=971
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 Water sample 
 blue % transmitted 
 green % transmitted 
 red % transmitted 
 
 
 1 
 + 1 tsp milk 
 13/13 = 100% 
 14/14 = 100% 
 12/12 = 100% 
 
 
 2 
 +1tsp milk + blue 
 10/13 = 77% 
 9/14 = 64% 
 6/12 = 50% 
 
 
 3 
 +1tsp milk +green 
 9/13 = 69% 
 13/14 = 93% 
 7/12 = 58% 
 
 
 4 
 +1tsp milk + tea 
 7/13 = 54% 
 9/14 = 69% 
 9/12 = 75%

time: 31 ms (started: 2023-12-16 17:33:06 +01:00)


### Export to xlsx

All tables from a document can be exported in xslx format, with one table per worksheet.<br>
Arguments are similar to the <code>extract_tables</code> method.

In [9]:
pdf = PDF(src="data/tables.pdf")

# Export to file
pdf.to_xlsx('data/tables.xlsx',
            ocr=tesseract_ocr,
            implicit_rows=False,
            borderless_tables=False,
            min_confidence=50)

time: 546 ms (started: 2023-12-16 17:33:10 +01:00)


In [10]:
for ws in load_workbook("data/tables.xlsx"):
    print(f"Worksheet {ws.title} : {len(tuple(ws.rows))} rows, {len(tuple(ws.rows)[0])} columns")

Worksheet Page 1 - Table 1 : 5 rows, 4 columns
Worksheet Page 1 - Table 2 : 5 rows, 4 columns
Worksheet Page 2 - Table 1 : 5 rows, 4 columns
Worksheet Page 2 - Table 2 : 5 rows, 4 columns
time: 16 ms (started: 2023-12-16 17:33:13 +01:00)
